In [10]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Read in parsed monthly data
df = pd.read_feather("crsp_monthly.ftr")
df

,permno,caldt,cusip,ticker,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr
0,10000,1986-01-31,68391610,OMFGA,10,3,3990,-4.37500,NaN,1771.0,3680.0,1.0
1,10000,1986-02-28,68391610,OMFGA,10,3,3990,-3.25000,-0.257143,828.0,3680.0,1.0
2,10000,1986-03-31,68391610,OMFGA,10,3,3990,-4.43750,0.365385,1078.0,3680.0,1.0
3,10000,1986-04-30,68391610,OMFGA,10,3,3990,-4.00000,-0.098592,957.0,3793.0,1.0
4,10000,1986-05-30,68391610,OMFGA,10,3,3990,-3.10938,-0.222656,1074.0,3793.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4889699,93436,2023-08-31,88160R10,TSLA,11,3,9999,258.07999,-0.034962,25029170.0,3173994.0,1.0
4889700,93436,2023-09-29,88160R10,TSLA,11,3,9999,250.22000,-0.030456,24395440.0,3179000.0,1.0
4889701,93436,2023-10-31,88160R10,TSLA,11,3,9999,200.84000,-0.197346,25905681.0,3178921.0,1.0
4889702,93436,2023-11-30,88160R10,TSLA,11,3,9999,240.08000,0.195379,26395792.0,3178921.0,1.0


In [12]:
# Cleaning

df['prc'] = abs(df['prc'])

df['ret'] = df.groupby('permno')['prc'].pct_change()

df.sort_values(by=['permno', 'caldt'], inplace=True)

df = df[['permno', 'caldt', 'prc', 'ret']]

df

,permno,caldt,prc,ret
0,10000,1986-01-31,4.37500,NaN
1,10000,1986-02-28,3.25000,-0.257143
2,10000,1986-03-31,4.43750,0.365385
3,10000,1986-04-30,4.00000,-0.098592
4,10000,1986-05-30,3.10938,-0.222655
...,...,...,...,...
4889699,93436,2023-08-31,258.07999,-0.034962
4889700,93436,2023-09-29,250.22000,-0.030456
4889701,93436,2023-10-31,200.84000,-0.197346
4889702,93436,2023-11-30,240.08000,0.195379


In [13]:
# Calculate momentum feature

# Returns
df['ret'] = df.groupby("permno")['prc'].pct_change()

# Log Returns
df['logret'] = np.log(1 + df['ret'])

# Momentum from t-12 to t-2
df['mom'] = df.groupby('permno')['logret'].rolling(11,11).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(2)

# Drop columns
df.drop(columns=['logret'], inplace=True)

df

C:\Users\andre\AppData\Local\Temp\ipykernel_28408\454987151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ret'] = df.groupby("permno")['prc'].pct_change()
C:\Users\andre\AppData\Local\Temp\ipykernel_28408\454987151.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['logret'] = np.log(1 + df['ret'])
C:\Users\andre\AppData\Local\Temp\ipykernel_28408\454987151.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,permno,caldt,prc,ret,mom
0,10000,1986-01-31,4.37500,NaN,NaN
1,10000,1986-02-28,3.25000,-0.257143,NaN
2,10000,1986-03-31,4.43750,0.365385,NaN
3,10000,1986-04-30,4.00000,-0.098592,NaN
4,10000,1986-05-30,3.10938,-0.222655,NaN
...,...,...,...,...,...
4889699,93436,2023-08-31,258.07999,-0.034962,-1.225383
4889700,93436,2023-09-29,250.22000,-0.030456,-0.030129
4889701,93436,2023-10-31,200.84000,-0.197346,-0.027403
4889702,93436,2023-11-30,240.08000,0.195379,0.095015


In [14]:
# Lagging and filtering price

df['prclag'] = df.groupby('permno')['prc'].shift(1)

df = df.query("mom == mom and prclag >= 5").reset_index(drop=True)

df

C:\Users\andre\AppData\Local\Temp\ipykernel_28408\3917052757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prclag'] = df.groupby('permno')['prc'].shift(1)


,permno,caldt,prc,ret,mom,prclag
0,10001,1987-02-27,6.25000,-0.074074,1.335314e-01,6.75000
1,10001,1987-03-31,6.37500,0.020000,7.696104e-02,6.25000
2,10001,1987-04-30,6.12500,-0.039216,-9.950331e-03,6.37500
3,10001,1987-05-29,5.68750,-0.071429,2.775558e-17,6.12500
4,10001,1987-06-30,5.87500,0.032967,-3.015304e-02,5.68750
...,...,...,...,...,...,...
3475549,93436,2023-08-31,258.07999,-0.034962,-1.225383e+00,267.42999
3475550,93436,2023-09-29,250.22000,-0.030456,-3.012897e-02,258.07999
3475551,93436,2023-10-31,200.84000,-0.197346,-2.740320e-02,250.22000
3475552,93436,2023-11-30,240.08000,0.195379,9.501453e-02,200.84000


In [15]:
# Momentum bins
df['mombins'] = df.groupby("caldt")['mom'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

df

,permno,caldt,prc,ret,mom,prclag,mombins
0,10001,1987-02-27,6.25000,-0.074074,1.335314e-01,6.75000,7
1,10001,1987-03-31,6.37500,0.020000,7.696104e-02,6.25000,6
2,10001,1987-04-30,6.12500,-0.039216,-9.950331e-03,6.37500,4
3,10001,1987-05-29,5.68750,-0.071429,2.775558e-17,6.12500,4
4,10001,1987-06-30,5.87500,0.032967,-3.015304e-02,5.68750,5
...,...,...,...,...,...,...,...
3475549,93436,2023-08-31,258.07999,-0.034962,-1.225383e+00,267.42999,0
3475550,93436,2023-09-29,250.22000,-0.030456,-3.012897e-02,258.07999,3
3475551,93436,2023-10-31,200.84000,-0.197346,-2.740320e-02,250.22000,2
3475552,93436,2023-11-30,240.08000,0.195379,9.501453e-02,200.84000,6


In [16]:
# Form portfolios on momentum and id bins
port = df.groupby(['caldt', 'mombins'])['ret'].mean().unstack(level=['mombins'])*100
port

mombins,0,1,2,3,4,5,6,7,8,9
caldt,,,,,,,,,,
1927-01-31,-5.000201,0.795712,12.909646,3.646013,-0.306076,2.180967,-1.182049,-0.297314,0.944535,-2.396638
1927-02-28,7.631359,4.861211,4.049099,8.727187,5.146907,3.376000,5.183943,4.115877,5.221954,5.220836
1927-03-31,-3.171003,-2.386555,-3.556249,-4.730523,-1.012228,-2.291559,-0.823128,-0.379411,-0.879730,-2.702720
1927-04-30,2.316999,-2.061002,-1.143159,-2.205979,0.214968,-0.420524,-2.820777,-2.158090,0.415903,1.972166
1927-05-31,2.493081,5.521735,5.845318,4.667633,9.274352,7.305392,6.040890,8.241498,4.972917,8.365446
...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-6.251307,-5.206314,-4.360173,-2.732700,-2.611836,-2.658270,-1.882625,-3.336755,-3.383923,-5.073977
2023-09-29,-6.392847,-5.547427,-4.463296,-3.011169,-3.196274,-2.889431,-3.762092,-4.434087,-5.873602,-7.313713
2023-10-31,-8.080046,-5.423299,-3.405207,-3.022879,-3.261059,-3.860035,-4.134102,-3.401447,-4.972149,-7.622270


In [17]:
# Summary
port['spread'] = port[9] - port[0]
summary(port).loc[['count','mean','std','tstat']].round(3)

mombins,0,1,2,3,4,5,6,7,8,9,spread
count,1164.000,1164.000,1164.000,1164.000,1164.000,1164.000,1164.000,1164.000,1164.000,1164.000,1164.000
mean,0.258,0.314,0.422,0.437,0.502,0.591,0.628,0.632,0.697,0.672,0.414
std,8.557,7.245,6.634,6.207,5.942,5.673,5.593,5.690,6.025,7.131,5.674
tstat,1.030,1.480,2.171,2.404,2.884,3.556,3.828,3.789,3.949,3.216,2.488
